# DataFrames

Starting to rock the world with Apache Spark 

## Create and Show a DF

In [1]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
df = spark.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])

In [3]:
df.count()

3

In [4]:
df.show()

+---+---+
|  A|  B|
+---+---+
|  1|  4|
|  2|  5|
|  3|  6|
+---+---+



## Select a column

In [5]:
#Returns Column Object
df.A

Column<b'A'>

In [6]:
df.select('A').show()

+---+
|  A|
+---+
|  1|
|  2|
|  3|
+---+



## Add a new Column based on another

In [7]:
#Adding a new column
df.withColumn('C',df.A+1).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  4|  2|
|  2|  5|  3|
|  3|  6|  4|
+---+---+---+



## Add a new Column with constant values

In [10]:
from pyspark.sql.functions import lit
df.withColumn('C',lit(5)).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  4|  5|
|  2|  5|  5|
|  3|  6|  5|
+---+---+---+



## Filter columns

In [11]:
df.select('A',(df.A > 2).alias("State")).show()

+---+-----+
|  A|State|
+---+-----+
|  1|false|
|  2|false|
|  3| true|
+---+-----+



In [12]:
df[(df.A > 2)].show()

+---+---+
|  A|  B|
+---+---+
|  3|  6|
+---+---+



## GroupBy

In [13]:
df = spark.createDataFrame([('a',33), ('b',11), ('a',22)],['names','age'])

In [14]:
df.show()

+-----+---+
|names|age|
+-----+---+
|    a| 33|
|    b| 11|
|    a| 22|
+-----+---+



In [15]:
gdf = df.groupBy(df.names)

In [18]:
gdf.agg({"*":"count"}).collect()

[Row(names='b', count(1)=1), Row(names='a', count(1)=2)]

In [20]:
from pyspark.sql import functions as F
df = spark.createDataFrame([('a',33), ('b',11), ('a',22)],['names','age'])
gdf = df.groupBy(df.names)

gdf.agg(F.min(df.age)).show()

+-----+--------+
|names|min(age)|
+-----+--------+
|    b|      11|
|    a|      22|
+-----+--------+



In [21]:
g2df = df.groupBy(df.names)
g2df.min('age').collect()

[Row(names='b', min(age)=11), Row(names='a', min(age)=22)]

### Generate your own DataFrame
Create `stringRDD` RDD and then convert it into a DataFrame when we're reading `stringJSONRDD` using `spark.read.json`.

In [22]:
# Generate our own JSON data 
string_JSON_RDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Argenis",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Liliana",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Ana",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [23]:
# Create DataFrame
swimmers_JSON = spark.read.json(string_JSON_RDD)

In [27]:
swimmers_JSON.show(2)

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|Argenis|
| 22|   green|234|Liliana|
+---+--------+---+-------+
only showing top 2 rows



In [28]:
# Create temporary table
swimmers_JSON.createOrReplaceTempView("swimmersJSON")

In [29]:
# DataFrame API
swimmers_JSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|Argenis|
| 22|   green|234|Liliana|
| 23|    blue|345|    Ana|
+---+--------+---+-------+



In [31]:
# SQL Query
spark.sql("select * from swimmersJSON").show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|Argenis|
| 22|   green|234|Liliana|
| 23|    blue|345|    Ana|
+---+--------+---+-------+



In [32]:
spark.sql("select * from swimmersJSON")

DataFrame[age: bigint, eyeColor: string, id: string, name: string]

## Inferring the Schema Using Reflection
Note that Apache Spark is inferring the schema using reflection; i.e. it automaticlaly determines the schema of the data based on reviewing the JSON data.

In [33]:
# Print the schema
swimmers_JSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



Notice that Spark was able to determine infer the schema (when reviewing the schema using `.printSchema`).

But what if we want to programmatically specify the schema?

## Programmatically Specifying the Schema
In this case, let's specify the schema for a `CSV` text file.

In [34]:
from pyspark.sql.types import *

string_CSV_RDD = sc.parallelize([(123, 'Argenis', 19, 'brown'), (234, 'Liliana', 22, 'green'), (345, 'Ana', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(string_CSV_RDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [35]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [36]:
spark.sql("select * from swimmers")

DataFrame[id: bigint, name: string, age: bigint, eyeColor: string]

As you can see from above, we can programmatically apply the `schema` instead of allowing the Spark engine to infer the schema via reflection.

Additional Resources include:
* [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
* [Spark SQL, DataFrames, and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema): This is in reference to Programmatically Specifying the Schema using a `CSV` file.

## Querying with dataframe

In [37]:
# Query id and age for swimmers with age = 22 via DataFrame API
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [38]:
# Query id and age for swimmers with age = 22 via DataFrame API in another way
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [39]:
# Query id and age for swimmers with age = 22 in SQL
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [40]:
spark.sql("select id, age from swimmers where age = 22")

DataFrame[id: bigint, age: bigint]

In [41]:
# Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+-------+--------+
|   name|eyeColor|
+-------+--------+
|Argenis|   brown|
|    Ana|    blue|
+-------+--------+



In [ ]:
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'")

## Querying with the DataFrame API
With DataFrames, you can start writing your queries using the DataFrame API

In [ ]:
# Show the values 
swimmers.show()

In [ ]:
# Get count of rows
swimmers.count()

In [ ]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()

In [ ]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

## DataFrame Queries
* Understanding explode, selectExpr

In [44]:
# import pyspark class Row from module sql
from pyspark.sql import *

# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('favio', 'vazquez', 'no-reply@iron-ai.com', 100000)
employee11 = Employee('favio', 'vazquez', 'no-reply@bbva.com', 200000)
employee2 = Employee('argenis', 'leon', 'no-reply@iron-ai.com', 300000)
employee3 = Employee('liliana', None, 'no-reply@iron-ai.com', 350000)
employee31 = Employee('liliana', None, 'no-reply@google.com', 180000)
employee4 = Employee(None, 'ferro', 'no-reply@iron-ai.com', 160000)

# Create the DepartmentWithEmployees instances from Departments and Employees
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4, employee11])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4, employee31])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

In [45]:
departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2]
df1 = spark.createDataFrame(departmentsWithEmployeesSeq1)

departmentsWithEmployeesSeq2 = [departmentWithEmployees3, departmentWithEmployees4]
df2 = spark.createDataFrame(departmentsWithEmployeesSeq2)

In [46]:
df1.show()

+--------------------+--------------------+
|          department|           employees|
+--------------------+--------------------+
|[123456,Computer ...|[[favio,vazquez,n...|
|[789012,Mechanica...|[[liliana,null,no...|
+--------------------+--------------------+



In [47]:
df2.show(truncate=False)

+--------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|department                |employees                                                                                                                         |
+--------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|[345678,Theater and Drama]|[[favio,vazquez,no-reply@iron-ai.com,100000], [null,ferro,no-reply@iron-ai.com,160000], [liliana,null,no-reply@google.com,180000]]|
|[901234,Indoor Recreation]|[[argenis,leon,no-reply@iron-ai.com,300000], [liliana,null,no-reply@iron-ai.com,350000]]                                          |
+--------------------------+----------------------------------------------------------------------------------------------------------------------------------+



In [48]:
unionDF = df1.union(df2)

In [49]:
unionDF.show()

+--------------------+--------------------+
|          department|           employees|
+--------------------+--------------------+
|[123456,Computer ...|[[favio,vazquez,n...|
|[789012,Mechanica...|[[liliana,null,no...|
|[345678,Theater a...|[[favio,vazquez,n...|
|[901234,Indoor Re...|[[argenis,leon,no...|
+--------------------+--------------------+



In [50]:
from pyspark.sql.functions import explode

df = unionDF.select("department",explode("employees").alias("e"))

In [51]:
df.show(truncate=False)

+-------------------------------+-------------------------------------------+
|department                     |e                                          |
+-------------------------------+-------------------------------------------+
|[123456,Computer Science]      |[favio,vazquez,no-reply@iron-ai.com,100000]|
|[123456,Computer Science]      |[argenis,leon,no-reply@iron-ai.com,300000] |
|[789012,Mechanical Engineering]|[liliana,null,no-reply@iron-ai.com,350000] |
|[789012,Mechanical Engineering]|[null,ferro,no-reply@iron-ai.com,160000]   |
|[789012,Mechanical Engineering]|[favio,vazquez,no-reply@bbva.com,200000]   |
|[345678,Theater and Drama]     |[favio,vazquez,no-reply@iron-ai.com,100000]|
|[345678,Theater and Drama]     |[null,ferro,no-reply@iron-ai.com,160000]   |
|[345678,Theater and Drama]     |[liliana,null,no-reply@google.com,180000]  |
|[901234,Indoor Recreation]     |[argenis,leon,no-reply@iron-ai.com,300000] |
|[901234,Indoor Recreation]     |[liliana,null,no-reply@iron-ai.

In [ ]:
df.collect()

In [52]:
df.selectExpr("department.id","department.name","e.firstName", "e.lastName", "e.email", "e.salary").show()

+------+--------------------+---------+--------+--------------------+------+
|    id|                name|firstName|lastName|               email|salary|
+------+--------------------+---------+--------+--------------------+------+
|123456|    Computer Science|    favio| vazquez|no-reply@iron-ai.com|100000|
|123456|    Computer Science|  argenis|    leon|no-reply@iron-ai.com|300000|
|789012|Mechanical Engine...|  liliana|    null|no-reply@iron-ai.com|350000|
|789012|Mechanical Engine...|     null|   ferro|no-reply@iron-ai.com|160000|
|789012|Mechanical Engine...|    favio| vazquez|   no-reply@bbva.com|200000|
|345678|   Theater and Drama|    favio| vazquez|no-reply@iron-ai.com|100000|
|345678|   Theater and Drama|     null|   ferro|no-reply@iron-ai.com|160000|
|345678|   Theater and Drama|  liliana|    null| no-reply@google.com|180000|
|901234|   Indoor Recreation|  argenis|    leon|no-reply@iron-ai.com|300000|
|901234|   Indoor Recreation|  liliana|    null|no-reply@iron-ai.com|350000|

In [53]:
from pyspark.sql.functions import explode

df = unionDF.select(explode("employees").alias("e"))

explodeDF = df.selectExpr("e.firstName", "e.lastName", "e.email", "e.salary")
explodeDF.show()

+---------+--------+--------------------+------+
|firstName|lastName|               email|salary|
+---------+--------+--------------------+------+
|    favio| vazquez|no-reply@iron-ai.com|100000|
|  argenis|    leon|no-reply@iron-ai.com|300000|
|  liliana|    null|no-reply@iron-ai.com|350000|
|     null|   ferro|no-reply@iron-ai.com|160000|
|    favio| vazquez|   no-reply@bbva.com|200000|
|    favio| vazquez|no-reply@iron-ai.com|100000|
|     null|   ferro|no-reply@iron-ai.com|160000|
|  liliana|    null| no-reply@google.com|180000|
|  argenis|    leon|no-reply@iron-ai.com|300000|
|  liliana|    null|no-reply@iron-ai.com|350000|
+---------+--------+--------------------+------+



In [54]:
filterDF = explodeDF.filter( explodeDF.firstName == 'favio').sort(explodeDF.salary)

In [55]:
filterDF.show()

+---------+--------+--------------------+------+
|firstName|lastName|               email|salary|
+---------+--------+--------------------+------+
|    favio| vazquez|no-reply@iron-ai.com|100000|
|    favio| vazquez|no-reply@iron-ai.com|100000|
|    favio| vazquez|   no-reply@bbva.com|200000|
+---------+--------+--------------------+------+



In [56]:
# Different ways of calling a column
from pyspark.sql.functions import *
filterDF = explodeDF.filter((filterDF.firstName == "favio") | (col("firstName") == "argenis")).sort(desc("lastName"))
filterDF.show()

+---------+--------+--------------------+------+
|firstName|lastName|               email|salary|
+---------+--------+--------------------+------+
|    favio| vazquez|   no-reply@bbva.com|200000|
|    favio| vazquez|no-reply@iron-ai.com|100000|
|    favio| vazquez|no-reply@iron-ai.com|100000|
|  argenis|    leon|no-reply@iron-ai.com|300000|
|  argenis|    leon|no-reply@iron-ai.com|300000|
+---------+--------+--------------------+------+



In [ ]:
whereDF = explodeDF.where((col("firstName") == "argenis") | (col("firstName") == "favio")).sort(asc("lastName"))
whereDF.show()

### Handling Missing Data

In [57]:
from pyspark.sql.functions import col, asc, desc
filterNonNullDF = explodeDF.filter(col("firstName").isNotNull()).filter(col("lastName").isNotNull()).sort("email")
filterNonNullDF.show()

+---------+--------+--------------------+------+
|firstName|lastName|               email|salary|
+---------+--------+--------------------+------+
|    favio| vazquez|   no-reply@bbva.com|200000|
|    favio| vazquez|no-reply@iron-ai.com|100000|
|  argenis|    leon|no-reply@iron-ai.com|300000|
|    favio| vazquez|no-reply@iron-ai.com|100000|
|  argenis|    leon|no-reply@iron-ai.com|300000|
+---------+--------+--------------------+------+



In [58]:
from pyspark.sql.functions import countDistinct,count

countDistinctDF = explodeDF.select("firstName", "lastName")\
  .groupBy("firstName", "lastName")\
  .agg(countDistinct("firstName"))

countDistinctDF.show()

+---------+--------+-------------------------+
|firstName|lastName|count(DISTINCT firstName)|
+---------+--------+-------------------------+
|  liliana|    null|                        1|
|     null|   ferro|                        0|
|    favio| vazquez|                        1|
|  argenis|    leon|                        1|
+---------+--------+-------------------------+



In [59]:
# Careful
from pyspark.sql.functions import count

countDistinctDF = explodeDF.select("firstName", "lastName")\
  .groupBy("firstName", "lastName")\
  .agg(count("*"))
countDistinctDF.show()

+---------+--------+--------+
|firstName|lastName|count(1)|
+---------+--------+--------+
|  argenis|    leon|       2|
|     null|   ferro|       2|
|  liliana|    null|       3|
|    favio| vazquez|       3|
+---------+--------+--------+



In [ ]:
explodeDF.describe("salary").show()

For more information, please refer to:
* [Spark SQL, DataFrames and Datasets Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#sql)
* [PySpark SQL Module: DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)
* [PySpark SQL Functions Module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)

### DropDuplicates

In [60]:
df = spark.createDataFrame([
        (1, 144.5, 5.9, 33, 'M'),
        (2, 167.2, 5.4, 45, 'M'),
        (3, 124.1, 5.2, 23, 'F'),
        (4, 144.5, 5.9, 33, 'M'),
        (5, 133.2, 5.7, 54, 'F'),
        (3, 124.1, 5.2, 23, 'F'),
        (5, 129.2, 5.3, 42, 'M'),
    ], ['id', 'weight', 'height', 'age', 'gender'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [61]:
df = df.dropDuplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [62]:
df.count()

6

In [ ]:
#Duplicates except for id column
df = df.dropDuplicates(subset=[c for c in df.columns if c != 'id'])

In [ ]:
df.show()

In [ ]:
[c for c in df.columns if c != 'id']

### Aggregation

In [ ]:
import pyspark.sql.functions as F
df.agg(
  F.count('id').alias('count'),
  F.countDistinct('id').alias('distinct')
).show()

### More on Handling Missing Data

In [ ]:
df_miss = spark.createDataFrame([
        (1, 143.5, 5.6, 28,   'M',  100000),
        (2, 167.2, 5.4, 45,   'M',  None),
        (3, None , 5.2, None, None, None),
        (4, 144.5, 5.9, 33,   'M',  None),
        (5, 133.2, 5.7, 54,   'F',  None),
        (6, 124.1, 5.2, None, 'F',  None),
        (7, 129.2, 5.3, 42,   'M',  76000),
    ], ['id', 'weight', 'height', 'age', 'gender', 'income'])

In [ ]:
df_miss.show()

In [ ]:
df_miss.printSchema()

In [ ]:
df_miss.describe().show()

In [ ]:
#Calculate missing columns for each row
df_miss.rdd.collect()

In [ ]:
df_miss.where('id == 3').show()

In [ ]:
import pyspark.sql.functions as F
df_miss.agg(
 F.count('weight'), F.count('height'), F.count('age'),F.count('gender'),F.count('income'),
 F.count('*')
).show()

In [ ]:
import pyspark.sql.functions as F
df_miss.agg(
 *[F.count(c)  for c in df.columns]
).show()

In [ ]:
df_miss.dropna(thresh=3).show()